In [159]:
import torch
torch.manual_seed(0)

dtype = torch.uint8
h, w = (4,4)
rgb = (3,4,9)
label = torch.randint(high=20, size=(h,w), dtype=dtype)
color = torch.tensor(rgb, dtype=dtype).unsqueeze(1)
print(color.size())
idx = 16
print(label)

color_mask = torch.randint(high=255, size=(3,h,w), dtype=dtype)
print(color_mask)

validx = label == torch.tensor((idx), dtype=dtype)
# validx = validx.repeat(3,1,1)
# print('Validx', validx)

print(color_mask[:, validx])
color_mask[:, validx] = color
# selected_values = color_mask[validx]
# print('Selected_values', selected_values)

color_mask
# print(selected_values)
# print(validx.dtype)

# selected_values = color
# print(selected_values)
# color_mask

torch.Size([3, 1])
tensor([[ 4, 19, 13,  0],
        [ 3, 19,  7,  3],
        [17,  3,  1,  6],
        [16, 19, 18, 16]], dtype=torch.uint8)
tensor([[[156, 108, 254, 198],
         [221,  44, 236, 209],
         [179, 206, 109,  59],
         [249, 150, 136, 192]],

        [[173, 135, 130, 225],
         [ 67, 104, 101, 103],
         [143, 123, 226,   4],
         [ 16, 227, 238,  25]],

        [[ 47,  61, 115, 149],
         [173,  31,  26, 185],
         [ 23, 196, 206,  47],
         [139,  91, 243, 129]]], dtype=torch.uint8)
tensor([[249, 192],
        [ 16,  25],
        [139, 129]], dtype=torch.uint8)


tensor([[[156, 108, 254, 198],
         [221,  44, 236, 209],
         [179, 206, 109,  59],
         [  3, 150, 136,   3]],

        [[173, 135, 130, 225],
         [ 67, 104, 101, 103],
         [143, 123, 226,   4],
         [  4, 227, 238,   4]],

        [[ 47,  61, 115, 149],
         [173,  31,  26, 185],
         [ 23, 196, 206,  47],
         [  5,  91, 243,   5]]], dtype=torch.uint8)

In [121]:
x = torch.rand((3,5,5))
y = torch.rand((1,5,5))

tensor([[[0.5023, 1.0234, 1.3719, 0.9549, 0.8926],
         [0.9254, 0.5992, 1.2001, 0.2866, 0.6278],
         [1.0996, 1.1748, 1.2150, 0.6607, 1.2522],
         [1.5924, 1.4105, 1.3366, 1.5906, 1.2740],
         [1.0309, 0.3776, 1.1572, 1.1933, 1.1658]],

        [[0.9555, 1.1448, 0.9743, 0.3217, 0.8711],
         [0.2774, 0.4277, 1.5511, 1.0544, 0.4756],
         [1.4515, 0.8644, 0.9316, 0.7280, 1.2935],
         [1.0218, 1.6569, 1.5704, 1.8326, 1.1508],
         [0.0560, 0.4386, 1.5901, 1.1014, 0.3614]],

        [[0.2325, 1.1740, 1.2117, 0.5384, 0.6992],
         [0.1139, 0.8847, 0.7303, 0.2886, 0.9468],
         [1.4330, 0.7438, 1.3106, 0.6526, 1.5549],
         [1.6606, 1.0520, 1.1931, 1.4973, 1.0368],
         [0.6694, 0.9592, 1.3704, 0.7528, 0.9881]]])